# Clustering: K-Means e DBSCAN

## Objetivos

- Compreender aprendizado não supervisionado
- Implementar e aplicar algoritmo K-Means
- Entender e usar DBSCAN para clustering baseado em densidade
- Avaliar qualidade de clusters
- Escolher número ótimo de clusters

## Pré-requisitos

- Conceitos básicos de ML
- Distâncias e métricas
- Visualização de dados


## 1. Introdução ao Clustering

**Clustering** é uma técnica de aprendizado não supervisionado que agrupa dados similares.

### Aplicações:

- **Segmentação de clientes**: Agrupar clientes por comportamento
- **Organização de documentos**: Agrupar textos por tópico
- **Análise de genes**: Identificar grupos de genes similares
- **Compressão de imagens**: Reduzir cores agrupando pixels similares

### Tipos de Clustering:

- **Particional**: K-Means, K-Medoids
- **Hierárquico**: Agglomerative, Divisive
- **Baseado em densidade**: DBSCAN, OPTICS
- **Baseado em modelo**: Gaussian Mixture Models


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.datasets import make_blobs, make_circles, make_moons
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    adjusted_rand_score,
    silhouette_score,
    calinski_harabasz_score,
    davies_bouldin_score,
)
from sklearn.decomposition import PCA
import warnings

warnings.filterwarnings("ignore")

# Configurar visualização
plt.style.use("default")
sns.set_palette("husl")
np.random.seed(42)

## 2. Algoritmo K-Means

### Funcionamento:

1. **Inicializar** K centroides aleatoriamente
2. **Atribuir** cada ponto ao centroide mais próximo
3. **Recalcular** centroides como média dos pontos atribuídos
4. **Repetir** até convergência

### Características:

- Clusters **esféricos** e de **tamanho similar**
- Sensível à **inicialização**
- Precisa definir **K** antecipadamente


In [ ]:
# Implementação manual do K-Means
class KMeansManual:
    def __init__(self, k=3, max_iters=100, random_state=42):
        self.k = k
        self.max_iters = max_iters
        self.random_state = random_state

    def fit(self, X):
        np.random.seed(self.random_state)

        # Inicializar centroides aleatoriamente
        n_samples, n_features = X.shape
        self.centroids = np.random.uniform(
            X.min(axis=0), X.max(axis=0), size=(self.k, n_features)
        )

        self.history = [self.centroids.copy()]

        for iteration in range(self.max_iters):
            # Atribuir pontos aos centroides mais próximos
            distances = np.sqrt(((X - self.centroids[:, np.newaxis]) ** 2).sum(axis=2))
            self.labels_ = np.argmin(distances, axis=0)

            # Atualizar centroides
            new_centroids = np.array(
                [
                    X[self.labels_ == i].mean(axis=0)
                    if np.any(self.labels_ == i)
                    else self.centroids[i]
                    for i in range(self.k)
                ]
            )

            # Verificar convergência
            if np.allclose(self.centroids, new_centroids):
                print(f"Convergiu em {iteration + 1} iterações")
                break

            self.centroids = new_centroids
            self.history.append(self.centroids.copy())

        return self

    def predict(self, X):
        distances = np.sqrt(((X - self.centroids[:, np.newaxis]) ** 2).sum(axis=2))
        return np.argmin(distances, axis=0)


# Criar dataset de exemplo
X_blobs, y_true = make_blobs(
    n_samples=300, centers=4, n_features=2, random_state=42, cluster_std=1.5
)

print(f"Dataset shape: {X_blobs.shape}")
print(f"Clusters verdadeiros: {np.unique(y_true)}")

# Visualizar dados originais
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.scatter(X_blobs[:, 0], X_blobs[:, 1], c=y_true, cmap="viridis", alpha=0.7)
plt.title("Dados Originais (com labels verdadeiros)")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")

plt.subplot(1, 2, 2)
plt.scatter(X_blobs[:, 0], X_blobs[:, 1], alpha=0.7, color="gray")
plt.title("Dados sem Labels (problema real)")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")

plt.tight_layout()
plt.show()

In [ ]:
# Aplicar K-Means manual
kmeans_manual = KMeansManual(k=4, random_state=42)
kmeans_manual.fit(X_blobs)

# Visualizar evolução do algoritmo
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

iterations_to_show = [0, 1, 2, 3, 4, len(kmeans_manual.history) - 1]

for i, iteration in enumerate(iterations_to_show):
    if iteration < len(kmeans_manual.history):
        centroids = kmeans_manual.history[iteration]

        # Calcular labels para esta iteração
        if iteration == 0:
            # Primeira iteração - só mostrar centroides iniciais
            axes[i].scatter(X_blobs[:, 0], X_blobs[:, 1], alpha=0.5, color="gray")
        else:
            # Calcular distâncias e labels
            distances = np.sqrt(((X_blobs - centroids[:, np.newaxis]) ** 2).sum(axis=2))
            labels = np.argmin(distances, axis=0)
            axes[i].scatter(
                X_blobs[:, 0], X_blobs[:, 1], c=labels, cmap="viridis", alpha=0.7
            )

        # Plotar centroides
        axes[i].scatter(
            centroids[:, 0], centroids[:, 1], c="red", marker="x", s=200, linewidths=3
        )
        axes[i].set_title(f"Iteração {iteration}")
        axes[i].set_xlabel("Feature 1")
        axes[i].set_ylabel("Feature 2")

plt.tight_layout()
plt.show()

# Comparar com sklearn
kmeans_sklearn = KMeans(n_clusters=4, random_state=42, n_init=10)
labels_sklearn = kmeans_sklearn.fit_predict(X_blobs)

# Avaliar resultados
ari_manual = adjusted_rand_score(y_true, kmeans_manual.labels_)
ari_sklearn = adjusted_rand_score(y_true, labels_sklearn)

print(f"Adjusted Rand Index:")
print(f"  Manual: {ari_manual:.3f}")
print(f"  Sklearn: {ari_sklearn:.3f}")

## 3. Escolhendo o Número de Clusters (K)

### Métodos para escolher K:

1. **Elbow Method**: Buscar "cotovelo" na curva de inércia
2. **Silhouette Score**: Medir qualidade da separação
3. **Gap Statistic**: Comparar com dados aleatórios
4. **Conhecimento do domínio**: Usar contexto do problema


In [ ]:
# Método do Cotovelo (Elbow Method)
def elbow_method(X, max_k=10):
    inertias = []
    k_range = range(1, max_k + 1)

    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X)
        inertias.append(kmeans.inertia_)

    return k_range, inertias


# Silhouette Score para diferentes valores de K
def silhouette_analysis(X, max_k=10):
    silhouette_scores = []
    k_range = range(2, max_k + 1)  # Silhouette precisa de pelo menos 2 clusters

    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = kmeans.fit_predict(X)
        score = silhouette_score(X, labels)
        silhouette_scores.append(score)

    return k_range, silhouette_scores


# Aplicar métodos
k_range_elbow, inertias = elbow_method(X_blobs, max_k=10)
k_range_sil, sil_scores = silhouette_analysis(X_blobs, max_k=10)

# Visualizar resultados
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Elbow Method
axes[0].plot(k_range_elbow, inertias, "bo-")
axes[0].set_xlabel("Número de Clusters (K)")
axes[0].set_ylabel("Inércia (WCSS)")
axes[0].set_title("Método do Cotovelo")
axes[0].grid(True, alpha=0.3)

# Destacar possível cotovelo
axes[0].axvline(x=4, color="red", linestyle="--", alpha=0.7, label="K=4 (cotovelo)")
axes[0].legend()

# Silhouette Score
axes[1].plot(k_range_sil, sil_scores, "ro-")
axes[1].set_xlabel("Número de Clusters (K)")
axes[1].set_ylabel("Silhouette Score")
axes[1].set_title("Análise de Silhouette")
axes[1].grid(True, alpha=0.3)

# Destacar melhor score
best_k = k_range_sil[np.argmax(sil_scores)]
axes[1].axvline(
    x=best_k, color="red", linestyle="--", alpha=0.7, label=f"Melhor K={best_k}"
)
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"Método do Cotovelo sugere: K=4 (análise visual)")
print(f"Silhouette Score sugere: K={best_k} (score={max(sil_scores):.3f})")
print(f"Clusters verdadeiros: {len(np.unique(y_true))}")

## 4. DBSCAN - Density-Based Clustering

### Vantagens do DBSCAN:

- **Não precisa definir K** antecipadamente
- **Detecta outliers** automaticamente
- **Clusters de formas arbitrárias**
- **Robusto a ruído**

### Parâmetros:

- **eps**: Distância máxima entre pontos do mesmo cluster
- **min_samples**: Número mínimo de pontos para formar cluster


In [ ]:
# Criar datasets com diferentes características
datasets = {
    "Blobs": make_blobs(
        n_samples=300, centers=4, n_features=2, random_state=42, cluster_std=1.5
    )[0],
    "Circles": make_circles(n_samples=300, noise=0.1, factor=0.5, random_state=42)[0],
    "Moons": make_moons(n_samples=300, noise=0.1, random_state=42)[0],
}

# Comparar K-Means vs DBSCAN
fig, axes = plt.subplots(3, 3, figsize=(15, 12))

for i, (name, X) in enumerate(datasets.items()):
    # Dados originais
    axes[i, 0].scatter(X[:, 0], X[:, 1], alpha=0.7, color="gray")
    axes[i, 0].set_title(f"{name} - Dados Originais")

    # K-Means
    kmeans = KMeans(n_clusters=2, random_state=42)
    kmeans_labels = kmeans.fit_predict(X)

    axes[i, 1].scatter(X[:, 0], X[:, 1], c=kmeans_labels, cmap="viridis", alpha=0.7)
    axes[i, 1].scatter(
        kmeans.cluster_centers_[:, 0],
        kmeans.cluster_centers_[:, 1],
        c="red",
        marker="x",
        s=200,
        linewidths=3,
    )
    axes[i, 1].set_title(f"{name} - K-Means")

    # DBSCAN
    # Ajustar parâmetros baseado no dataset
    if name == "Blobs":
        eps, min_samples = 1.5, 5
    elif name == "Circles":
        eps, min_samples = 0.3, 5
    else:  # Moons
        eps, min_samples = 0.3, 5

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    dbscan_labels = dbscan.fit_predict(X)

    # Pontos de ruído (outliers) têm label -1
    unique_labels = set(dbscan_labels)
    colors = plt.cm.viridis(np.linspace(0, 1, len(unique_labels)))

    for label, color in zip(unique_labels, colors):
        if label == -1:
            # Outliers em preto
            mask = dbscan_labels == label
            axes[i, 2].scatter(
                X[mask, 0],
                X[mask, 1],
                c="black",
                marker="x",
                s=50,
                alpha=0.7,
                label="Outliers",
            )
        else:
            mask = dbscan_labels == label
            axes[i, 2].scatter(
                X[mask, 0], X[mask, 1], c=[color], alpha=0.7, label=f"Cluster {label}"
            )

    axes[i, 2].set_title(f"{name} - DBSCAN (eps={eps}, min_samples={min_samples})")

    # Calcular métricas (quando possível)
    if len(np.unique(kmeans_labels)) > 1:
        kmeans_sil = silhouette_score(X, kmeans_labels)
        axes[i, 1].text(
            0.05,
            0.95,
            f"Silhouette: {kmeans_sil:.3f}",
            transform=axes[i, 1].transAxes,
            fontsize=10,
            bbox=dict(boxstyle="round", facecolor="white", alpha=0.8),
        )

    if len(np.unique(dbscan_labels)) > 1 and -1 not in dbscan_labels:
        dbscan_sil = silhouette_score(X, dbscan_labels)
        axes[i, 2].text(
            0.05,
            0.95,
            f"Silhouette: {dbscan_sil:.3f}",
            transform=axes[i, 2].transAxes,
            fontsize=10,
            bbox=dict(boxstyle="round", facecolor="white", alpha=0.8),
        )

    # Mostrar número de clusters encontrados
    n_clusters_kmeans = len(np.unique(kmeans_labels))
    n_clusters_dbscan = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
    n_outliers = list(dbscan_labels).count(-1)

    axes[i, 1].text(
        0.05,
        0.85,
        f"Clusters: {n_clusters_kmeans}",
        transform=axes[i, 1].transAxes,
        fontsize=10,
        bbox=dict(boxstyle="round", facecolor="lightblue", alpha=0.8),
    )

    axes[i, 2].text(
        0.05,
        0.85,
        f"Clusters: {n_clusters_dbscan}\nOutliers: {n_outliers}",
        transform=axes[i, 2].transAxes,
        fontsize=10,
        bbox=dict(boxstyle="round", facecolor="lightgreen", alpha=0.8),
    )

# Remover labels dos eixos para clareza
for ax in axes.flat:
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
plt.show()

## 5. Otimizando Parâmetros do DBSCAN


In [ ]:
# Método para escolher eps: k-distance plot
def k_distance_plot(X, k=5):
    """Plota k-distance para ajudar a escolher eps no DBSCAN"""
    from sklearn.neighbors import NearestNeighbors

    # Encontrar k vizinhos mais próximos
    neighbors = NearestNeighbors(n_neighbors=k)
    neighbors.fit(X)
    distances, indices = neighbors.kneighbors(X)

    # Pegar distância para o k-ésimo vizinho
    k_distances = distances[:, k - 1]
    k_distances = np.sort(k_distances, axis=0)

    plt.figure(figsize=(10, 6))
    plt.plot(range(len(k_distances)), k_distances, "b-")
    plt.xlabel("Pontos ordenados por distância")
    plt.ylabel(f"{k}-distance")
    plt.title(f'K-Distance Plot (k={k})\nProcure por "cotovelo" para escolher eps')
    plt.grid(True, alpha=0.3)

    # Sugerir eps baseado no "cotovelo"
    # Método simples: encontrar maior mudança na segunda derivada
    if len(k_distances) > 10:
        # Calcular segunda derivada
        second_derivative = np.diff(k_distances, n=2)
        elbow_idx = np.argmax(second_derivative) + 2  # +2 devido ao diff duplo
        suggested_eps = k_distances[elbow_idx]

        plt.axhline(
            y=suggested_eps,
            color="red",
            linestyle="--",
            label=f"Eps sugerido: {suggested_eps:.3f}",
        )
        plt.axvline(x=elbow_idx, color="red", linestyle="--", alpha=0.5)
        plt.legend()

    plt.show()

    return k_distances


# Aplicar k-distance plot para dataset de círculos
X_circles = datasets["Circles"]
k_distances = k_distance_plot(X_circles, k=5)

# Testar diferentes valores de eps
eps_values = [0.1, 0.2, 0.3, 0.4, 0.5]
min_samples = 5

fig, axes = plt.subplots(1, len(eps_values), figsize=(20, 4))

for i, eps in enumerate(eps_values):
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(X_circles)

    # Contar clusters e outliers
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_outliers = list(labels).count(-1)

    # Plotar
    unique_labels = set(labels)
    colors = plt.cm.viridis(np.linspace(0, 1, len(unique_labels)))

    for label, color in zip(unique_labels, colors):
        if label == -1:
            mask = labels == label
            axes[i].scatter(
                X_circles[mask, 0],
                X_circles[mask, 1],
                c="black",
                marker="x",
                s=50,
                alpha=0.7,
            )
        else:
            mask = labels == label
            axes[i].scatter(
                X_circles[mask, 0], X_circles[mask, 1], c=[color], alpha=0.7
            )

    axes[i].set_title(f"eps={eps}\nClusters: {n_clusters}, Outliers: {n_outliers}")
    axes[i].set_xticks([])
    axes[i].set_yticks([])

plt.suptitle("Efeito do Parâmetro eps no DBSCAN", fontsize=16)
plt.tight_layout()
plt.show()

## 6. Métricas de Avaliação de Clustering


In [ ]:
# Função para avaliar múltiplas métricas
def evaluate_clustering(X, labels, true_labels=None):
    """Avalia clustering com múltiplas métricas"""
    results = {}

    # Métricas que não precisam de labels verdadeiros
    if len(set(labels)) > 1 and -1 not in labels:
        results["Silhouette Score"] = silhouette_score(X, labels)
        results["Calinski-Harabasz Index"] = calinski_harabasz_score(X, labels)
        results["Davies-Bouldin Index"] = davies_bouldin_score(X, labels)

    # Métricas que precisam de labels verdadeiros
    if true_labels is not None:
        results["Adjusted Rand Index"] = adjusted_rand_score(true_labels, labels)

    # Informações básicas
    results["N Clusters"] = len(set(labels)) - (1 if -1 in labels else 0)
    results["N Outliers"] = list(labels).count(-1)
    results["N Samples"] = len(labels)

    return results


# Comparar algoritmos em dataset com labels conhecidos
X_test, y_test = make_blobs(
    n_samples=300, centers=3, n_features=2, random_state=42, cluster_std=1.0
)

# Normalizar dados (importante para alguns algoritmos)
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

# Aplicar diferentes algoritmos
algorithms = {
    "K-Means (K=3)": KMeans(n_clusters=3, random_state=42),
    "K-Means (K=2)": KMeans(n_clusters=2, random_state=42),
    "K-Means (K=4)": KMeans(n_clusters=4, random_state=42),
    "DBSCAN (eps=0.5)": DBSCAN(eps=0.5, min_samples=5),
    "DBSCAN (eps=0.3)": DBSCAN(eps=0.3, min_samples=5),
}

results_comparison = {}

for name, algorithm in algorithms.items():
    if "DBSCAN" in name:
        labels = algorithm.fit_predict(
            X_test_scaled
        )  # DBSCAN se beneficia de normalização
    else:
        labels = algorithm.fit_predict(X_test)

    results_comparison[name] = evaluate_clustering(X_test, labels, y_test)

# Criar DataFrame para comparação
comparison_df = pd.DataFrame(results_comparison).T
print("Comparação de Algoritmos de Clustering:")
print("=" * 60)
print(comparison_df.round(3))

# Visualizar resultados
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# Dados originais
axes[0].scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap="viridis", alpha=0.7)
axes[0].set_title("Labels Verdadeiros")

# Resultados dos algoritmos
for i, (name, algorithm) in enumerate(algorithms.items(), 1):
    if "DBSCAN" in name:
        labels = algorithm.fit_predict(X_test_scaled)
        X_plot = X_test_scaled
    else:
        labels = algorithm.fit_predict(X_test)
        X_plot = X_test

    # Plotar clusters
    unique_labels = set(labels)
    colors = plt.cm.viridis(np.linspace(0, 1, len(unique_labels)))

    for label, color in zip(unique_labels, colors):
        if label == -1:
            mask = labels == label
            axes[i].scatter(
                X_plot[mask, 0], X_plot[mask, 1], c="black", marker="x", s=50, alpha=0.7
            )
        else:
            mask = labels == label
            axes[i].scatter(X_plot[mask, 0], X_plot[mask, 1], c=[color], alpha=0.7)

    # Adicionar centroides para K-Means
    if hasattr(algorithm, "cluster_centers_"):
        axes[i].scatter(
            algorithm.cluster_centers_[:, 0],
            algorithm.cluster_centers_[:, 1],
            c="red",
            marker="x",
            s=200,
            linewidths=3,
        )

    axes[i].set_title(
        f'{name}\nARI: {results_comparison[name]["Adjusted Rand Index"]:.3f}'
    )
    axes[i].set_xticks([])
    axes[i].set_yticks([])

plt.tight_layout()
plt.show()

## 7. Aplicação Prática: Segmentação de Clientes


In [ ]:
# Criar dataset sintético de clientes
np.random.seed(42)
n_customers = 1000

# Definir segmentos verdadeiros
segments = {
    "Jovens Econômicos": {
        "age": (18, 30),
        "income": (20000, 40000),
        "spending": (0.3, 0.6),
    },
    "Famílias Classe Média": {
        "age": (30, 50),
        "income": (40000, 80000),
        "spending": (0.4, 0.7),
    },
    "Seniors Altos Gastos": {
        "age": (50, 70),
        "income": (60000, 120000),
        "spending": (0.6, 0.9),
    },
    "Executivos": {"age": (25, 45), "income": (80000, 150000), "spending": (0.5, 0.8)},
}

# Gerar dados
customers_data = []
true_segments = []

for segment_name, params in segments.items():
    n_segment = n_customers // len(segments)

    ages = np.random.uniform(params["age"][0], params["age"][1], n_segment)
    incomes = np.random.uniform(params["income"][0], params["income"][1], n_segment)
    spending_ratios = np.random.uniform(
        params["spending"][0], params["spending"][1], n_segment
    )

    # Spending score baseado em renda e ratio
    spending_scores = (incomes * spending_ratios) / 1000  # Escala de 0-150

    for i in range(n_segment):
        customers_data.append([ages[i], incomes[i], spending_scores[i]])
        true_segments.append(segment_name)

# Converter para arrays
X_customers = np.array(customers_data)
true_segments = np.array(true_segments)

# Criar DataFrame
customers_df = pd.DataFrame(X_customers, columns=["Age", "Income", "Spending_Score"])
customers_df["True_Segment"] = true_segments

print("Dataset de Clientes:")
print(customers_df.head())
print(f"\nShape: {customers_df.shape}")
print(f"\nSegmentos verdadeiros:")
print(customers_df["True_Segment"].value_counts())

# Visualizar dados
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Age vs Income
scatter = axes[0, 0].scatter(
    customers_df["Age"],
    customers_df["Income"],
    c=pd.Categorical(customers_df["True_Segment"]).codes,
    cmap="viridis",
    alpha=0.7,
)
axes[0, 0].set_xlabel("Age")
axes[0, 0].set_ylabel("Income")
axes[0, 0].set_title("Age vs Income (por segmento verdadeiro)")

# Income vs Spending
axes[0, 1].scatter(
    customers_df["Income"],
    customers_df["Spending_Score"],
    c=pd.Categorical(customers_df["True_Segment"]).codes,
    cmap="viridis",
    alpha=0.7,
)
axes[0, 1].set_xlabel("Income")
axes[0, 1].set_ylabel("Spending Score")
axes[0, 1].set_title("Income vs Spending Score")

# Age vs Spending
axes[1, 0].scatter(
    customers_df["Age"],
    customers_df["Spending_Score"],
    c=pd.Categorical(customers_df["True_Segment"]).codes,
    cmap="viridis",
    alpha=0.7,
)
axes[1, 0].set_xlabel("Age")
axes[1, 0].set_ylabel("Spending Score")
axes[1, 0].set_title("Age vs Spending Score")

# Distribuições
for segment in customers_df["True_Segment"].unique():
    segment_data = customers_df[customers_df["True_Segment"] == segment]
    axes[1, 1].hist(segment_data["Spending_Score"], alpha=0.7, label=segment, bins=20)

axes[1, 1].set_xlabel("Spending Score")
axes[1, 1].set_ylabel("Frequência")
axes[1, 1].set_title("Distribuição de Spending Score por Segmento")
axes[1, 1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Aplicar clustering nos dados de clientes
X_customers_numeric = customers_df[["Age", "Income", "Spending_Score"]].values

# Normalizar dados
scaler = StandardScaler()
X_customers_scaled = scaler.fit_transform(X_customers_numeric)

# Encontrar número ótimo de clusters
k_range, inertias = elbow_method(X_customers_scaled, max_k=8)
k_range_sil, sil_scores = silhouette_analysis(X_customers_scaled, max_k=8)

# Plotar análises
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(k_range, inertias, "bo-")
axes[0].set_xlabel("K")
axes[0].set_ylabel("Inércia")
axes[0].set_title("Elbow Method - Clientes")
axes[0].grid(True)

axes[1].plot(k_range_sil, sil_scores, "ro-")
axes[1].set_xlabel("K")
axes[1].set_ylabel("Silhouette Score")
axes[1].set_title("Silhouette Analysis - Clientes")
axes[1].grid(True)

plt.tight_layout()
plt.show()

# Aplicar K-Means e DBSCAN
optimal_k = 4  # Baseado no conhecimento do problema

# K-Means
kmeans_customers = KMeans(n_clusters=optimal_k, random_state=42)
kmeans_labels = kmeans_customers.fit_predict(X_customers_scaled)

# DBSCAN
dbscan_customers = DBSCAN(eps=0.5, min_samples=10)
dbscan_labels = dbscan_customers.fit_predict(X_customers_scaled)

# Avaliar resultados
true_labels_encoded = pd.Categorical(true_segments).codes

kmeans_results = evaluate_clustering(
    X_customers_scaled, kmeans_labels, true_labels_encoded
)
dbscan_results = evaluate_clustering(
    X_customers_scaled, dbscan_labels, true_labels_encoded
)

print("Resultados da Segmentação de Clientes:")
print("=" * 50)
print("K-Means:")
for metric, value in kmeans_results.items():
    print(
        f"  {metric}: {value:.3f}"
        if isinstance(value, float)
        else f"  {metric}: {value}"
    )

print("\nDBSCAN:")
for metric, value in dbscan_results.items():
    print(
        f"  {metric}: {value:.3f}"
        if isinstance(value, float)
        else f"  {metric}: {value}"
    )

# Visualizar segmentação
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Segmentos verdadeiros
axes[0].scatter(
    customers_df["Income"],
    customers_df["Spending_Score"],
    c=true_labels_encoded,
    cmap="viridis",
    alpha=0.7,
)
axes[0].set_xlabel("Income")
axes[0].set_ylabel("Spending Score")
axes[0].set_title("Segmentos Verdadeiros")

# K-Means
axes[1].scatter(
    customers_df["Income"],
    customers_df["Spending_Score"],
    c=kmeans_labels,
    cmap="viridis",
    alpha=0.7,
)
axes[1].set_xlabel("Income")
axes[1].set_ylabel("Spending Score")
axes[1].set_title(f'K-Means (ARI: {kmeans_results["Adjusted Rand Index"]:.3f})')

# DBSCAN
scatter = axes[2].scatter(
    customers_df["Income"],
    customers_df["Spending_Score"],
    c=dbscan_labels,
    cmap="viridis",
    alpha=0.7,
)
axes[2].set_xlabel("Income")
axes[2].set_ylabel("Spending Score")
axes[2].set_title(f'DBSCAN (ARI: {dbscan_results["Adjusted Rand Index"]:.3f})')

plt.tight_layout()
plt.show()

# Análise dos clusters encontrados pelo K-Means
customers_df["Cluster_KMeans"] = kmeans_labels
cluster_summary = customers_df.groupby("Cluster_KMeans")[
    ["Age", "Income", "Spending_Score"]
].agg(["mean", "std"])

print("\nResumo dos Clusters (K-Means):")
print("=" * 50)
print(cluster_summary.round(2))

## 8. Resumo e Boas Práticas

### Escolha do Algoritmo:

**K-Means:**

- ✅ Clusters esféricos e de tamanho similar
- ✅ Rápido e simples
- ✅ Boa interpretabilidade (centroides)
- ❌ Precisa definir K antecipadamente
- ❌ Sensível a outliers e ruído

**DBSCAN:**

- ✅ Detecta clusters de forma arbitrária
- ✅ Robusto a outliers
- ✅ Não precisa definir número de clusters
- ❌ Sensível aos parâmetros eps e min_samples
- ❌ Dificuldade com clusters de densidade variável

### Processo Recomendado:

1. **Explorar dados**: Visualizar, identificar padrões
2. **Pré-processar**: Normalizar, tratar outliers
3. **Escolher K**: Elbow method, Silhouette analysis
4. **Aplicar algoritmos**: Testar K-Means e DBSCAN
5. **Avaliar**: Métricas internas e externas
6. **Interpretar**: Dar significado aos clusters
7. **Validar**: Verificar com conhecimento do domínio
